In [101]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
import re

In [9]:
cd '/Users/augustincablant/Documents/GitHub/Viager'

/Users/augustincablant/Documents/GitHub/Viager


In [92]:
#Téléchargement du dataset des vendeurs 
df = pd.read_csv('Data/seller.csv')
df.head()

,Unnamed: 0,X,b_sexe,bcode,b_ville,b_pays,dcode,acte,b_nom,b_prenoms,...,tranche_age,age_acte,dec1,dec2,dec3,sexe_homme,idf,etranger,une_tete,type_libre
0,0,123,2,29279,TAULE,NaN,75116,189,ABOMES,PAULE MARIE,...,2,75,0,0,1,0,0,0,1,1
1,1,586,2,99107,RIGA,LETTONIE,75108,000000149,ABRAMA,MONIKA,...,1,61,0,0,0,0,0,1,1,0
2,2,614,1,87085,LIMOGES,NaN,92051,00203,ADAM,ETIENNE JACQUES MARIE,...,4,90,1,0,0,1,0,0,1,0
3,3,883,2,99114,BUCAREST,ROUMANIE,75116,000001222,ADANIA,SOFIA,...,3,77,0,1,0,0,0,1,1,0
4,4,1452,1,75108,PARIS 8,NaN,75113,00171,AKAR,ETIENNE DAVID,...,3,78,0,1,0,1,1,0,0,0


In [96]:
df_travail = df[['b_sexe',
                'dateA',
                'dateN',
                'dateD']]
df_travail['deces_theorique'] = 0

/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_56031/1229763422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_travail['deces_theorique'] = 0


In [97]:
def from_num_to_date(row):
    date_reference = datetime(1960, 1, 1)
    nombre_de_jours = row 
    date = date_reference + timedelta(days=nombre_de_jours)
    return date

df_travail['date_acte'] = df_travail['dateA'].apply(from_num_to_date)
df_travail['date_deces'] = df_travail['dateD'].apply(from_num_to_date)
df_travail.head()

/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_56031/3620545501.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_travail['date_acte'] = df_travail['dateA'].apply(from_num_to_date)
/var/folders/bt/r1dg46ys21lb57m4n4x11m6c0000gn/T/ipykernel_56031/3620545501.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_travail['date_deces'] = df_travail['dateD'].apply(from_num_to_date)


,b_sexe,dateA,dateN,dateD,deces_theorique,date_acte,date_deces
0,2,12774,-14750,16485,0,1994-12-22,2005-02-18
1,2,9645,-12643,15592,0,1986-05-29,2002-09-09
2,1,12260,-20582,13568,0,1993-07-26,1997-02-23
3,2,11442,-16541,15680,0,1991-04-30,2002-12-06
4,1,10777,-17541,13534,0,1989-07-04,1997-01-20


In [107]:
#Téléchargement des données de l'INSEE
insee = pd.read_excel('https://www.insee.fr/fr/statistiques/fichier/2416631/demo-deces-esp-vie-age-2.xlsx')
def function_insee(dataframe):
    columns = ["Espérance de vie à divers âges hommes"] + [int(re.findall(r'\d+', element)[0]) for element in list(dataframe[3:4].values[0])[1:]]
    dataframe = dataframe[3:]
    dataframe.columns = columns 
    dataframe = dataframe[1:]
    dataframe["Espérance de vie à divers âges femmes"] = list(dataframe.iloc[:,0].values)
    insee_hommes = dataframe.iloc[:, 0:7]
    insee_femmes = dataframe.iloc[:, 7:]
    return insee_hommes, insee_femmes
insee_hommes, insee_femmes = function_insee(insee)

In [88]:
colonnes = [0, 1, 20, 40, 60, 65]
for individu in df_travail.index.to_list():
    sexe = df_travail['b_sexe'][individu]
    if sexe == 1:
        dataframe = insee_hommes
    else:
        dataframe = insee_femmes
    date_acte = df_travail['date_acte'][individu]
    age_acte = (df_travail['dateA'][individu] - df_travail['dateN'][individu]) / 365
    for i, col in enumerate(colonnes):
        if age_acte <= col:
            age_theorique = dataframe.iloc[date_acte.year, str(age_acte)]
            continue


,Espérance de vie à divers âges hommes,à 0 an,à 1 an,à 20 ans,à 40 ans,à 60 ans,à 65 ans
4,2023,85.7,85.1,66.2,46.6,27.9,23.6
5,2022,85.1,84.4,65.6,46,27.3,23
6,2021,85.2,84.5,65.7,46,27.4,23.1
7,2020,85.1,84.4,65.5,45.9,27.3,22.9
8,2019,85.6,84.8,66,46.4,27.8,23.4
9,2018,85.4,84.7,65.9,46.3,27.7,23.3
10,2017,85.3,84.6,65.8,46.1,27.6,23.2
11,2016,85.3,84.6,65.7,46.1,27.6,23.2
12,2015,85.1,84.4,65.6,45.9,27.4,23
13,2014 y c. Mayotte,85.4,84.7,65.8,46.2,27.7,23.3


In [108]:
insee_hommes

,Espérance de vie à divers âges hommes,0,1,20,40,60,65
4,2023,85.7,85.1,66.2,46.6,27.9,23.6
5,2022,85.1,84.4,65.6,46,27.3,23
6,2021,85.2,84.5,65.7,46,27.4,23.1
7,2020,85.1,84.4,65.5,45.9,27.3,22.9
8,2019,85.6,84.8,66,46.4,27.8,23.4
9,2018,85.4,84.7,65.9,46.3,27.7,23.3
10,2017,85.3,84.6,65.8,46.1,27.6,23.2
11,2016,85.3,84.6,65.7,46.1,27.6,23.2
12,2015,85.1,84.4,65.6,45.9,27.4,23
13,2014 y c. Mayotte,85.4,84.7,65.8,46.2,27.7,23.3
